## Project 1: NYC Resteraunt Violations Cleaning Up
### Jacob Minkin

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
sns.set_style('darkgrid')

%matplotlib inline

## Set-Up

In [257]:
#Import Data
df = pd.read_csv('../data/Inspection.csv')

#Change Name of Column for easier searching
df.rename(columns = {'VIOLATION CODE':'CODE'}, inplace = True)

# Convert date into pandas date/time format
df['date'] = pd.to_datetime(df['INSPECTION DATE'])

df['Critical'] = np.where(df['CRITICAL FLAG'] == 'Critical', 1, 0)


In [282]:
Name_list = df.CAMIS.unique()
latest_inspection_dicts = {}

for name in Name_list:
   latest_inspection_dicts[name] = df[df.CAMIS == name].date.max()

In [259]:
df['isLatest'] = pd.to_datetime(df['INSPECTION DATE']) == df.Latest_Inspection

AttributeError: 'DataFrame' object has no attribute 'Latest_Inspection'

In [260]:
df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE,INSPECTION DATE,ACTION,...,BBL,NTA,Location Point,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts,date,Critical
0,50107551,Zazzys,Manhattan,73,GREENWICH AVENUE,10014.0,6465945909,NaN,1/1/1900,NaN,...,1.006140e+09,MN23,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
1,50131697,NaN,Brooklyn,804,GRAND STREET,11211.0,3478078023,NaN,1/1/1900,NaN,...,3.027900e+09,BK90,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
2,50123820,787 COFFEE,Manhattan,310,WEST 14 STREET,10014.0,9082308846,NaN,1/1/1900,NaN,...,1.006290e+09,MN23,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
3,50119514,PEBBLE BAR,Manhattan,1240,AVE OF TH AMER,NaN,7189098636,NaN,1/1/1900,NaN,...,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
4,50127198,Verdes Italian Eatery,Staten Island,4085,VICTORY BOULEVARD,10314.0,6313030934,NaN,1/1/1900,NaN,...,5.027710e+09,SI05,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209574,50052840,US FRIED CHICKEN & PIZZA,Brooklyn,129,DWIGHT STREET,11231.0,7186434662,Chicken,6/22/2022,Violations were cited in the following area(s).,...,3.005780e+09,BK33,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-22,1
209575,50072454,DUNKIN,Manhattan,381,CANAL STREET,10013.0,2126191222,Donuts,8/22/2022,Violations were cited in the following area(s).,...,1.002270e+09,MN24,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-22,1
209576,50093635,DOMINOS,Brooklyn,1544,BROADWAY,11221.0,5166038816,Pizza,8/8/2022,Violations were cited in the following area(s).,...,3.014920e+09,BK79,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-08,0
209577,40365882,NEW YORK PIZZA SUPREMA,Manhattan,413,8 AVENUE,10001.0,2125948939,Pizza,2/8/2022,Violations were cited in the following area(s).,...,1.007540e+09,MN13,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-08,1


In [261]:
df.Latest_Inspection = df.CAMIS.map(latest_inspection_dicts)
df['LatestandCrit'] = (df.Critical==1)  & (df.isLatest ==True )
df['NotLatestandCrit'] = (df.Critical==1)  & (df.isLatest ==False )

C:\Users\logic_switch\AppData\Local\Temp\ipykernel_14384\3883454151.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.Latest_Inspection = df.CAMIS.map(latest_inspection_dicts)


AttributeError: 'DataFrame' object has no attribute 'isLatest'

In [262]:
df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE,INSPECTION DATE,ACTION,...,BBL,NTA,Location Point,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts,date,Critical
0,50107551,Zazzys,Manhattan,73,GREENWICH AVENUE,10014.0,6465945909,NaN,1/1/1900,NaN,...,1.006140e+09,MN23,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
1,50131697,NaN,Brooklyn,804,GRAND STREET,11211.0,3478078023,NaN,1/1/1900,NaN,...,3.027900e+09,BK90,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
2,50123820,787 COFFEE,Manhattan,310,WEST 14 STREET,10014.0,9082308846,NaN,1/1/1900,NaN,...,1.006290e+09,MN23,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
3,50119514,PEBBLE BAR,Manhattan,1240,AVE OF TH AMER,NaN,7189098636,NaN,1/1/1900,NaN,...,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
4,50127198,Verdes Italian Eatery,Staten Island,4085,VICTORY BOULEVARD,10314.0,6313030934,NaN,1/1/1900,NaN,...,5.027710e+09,SI05,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209574,50052840,US FRIED CHICKEN & PIZZA,Brooklyn,129,DWIGHT STREET,11231.0,7186434662,Chicken,6/22/2022,Violations were cited in the following area(s).,...,3.005780e+09,BK33,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-22,1
209575,50072454,DUNKIN,Manhattan,381,CANAL STREET,10013.0,2126191222,Donuts,8/22/2022,Violations were cited in the following area(s).,...,1.002270e+09,MN24,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-22,1
209576,50093635,DOMINOS,Brooklyn,1544,BROADWAY,11221.0,5166038816,Pizza,8/8/2022,Violations were cited in the following area(s).,...,3.014920e+09,BK79,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-08,0
209577,40365882,NEW YORK PIZZA SUPREMA,Manhattan,413,8 AVENUE,10001.0,2125948939,Pizza,2/8/2022,Violations were cited in the following area(s).,...,1.007540e+09,MN13,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-08,1


In [263]:
df.Critical

0         0
1         0
2         0
3         0
4         0
         ..
209574    1
209575    1
209576    0
209577    1
209578    0
Name: Critical, Length: 209579, dtype: int32

In [264]:
df.iloc[14].date == latest_inspection_dicts[df.iloc[14].CAMIS]
df.iloc[14].Latest_Inspection

AttributeError: 'Series' object has no attribute 'Latest_Inspection'

In [265]:
latest_dicts = {}
for indx in range(1,len(df.CAMIS)):
   if df.iloc[indx].date == latest_inspection_dicts[df.iloc[indx].CAMIS]:
      df.iloc[indx].Latest_Inspection = 1


make a new col called latest inspection which includes the latest inspection for the resteraunt

In [266]:
df.iloc[2].date

Timestamp('1900-01-01 00:00:00')

In [267]:
df['Latest_Inspection'] = 0
df.Latest_Inspection


0        1900-01-01
1        1900-01-01
2        1900-01-01
3        1900-01-01
4        1900-01-01
            ...    
209574   2022-11-28
209575   2023-01-13
209576   2022-08-08
209577   2023-01-26
209578   2022-06-15
Name: CAMIS, Length: 209579, dtype: datetime64[ns]

In [268]:
#df.iloc[5].date
df[df.CAMIS == df.iloc[3].CAMIS].date.max()

Timestamp('1900-01-01 00:00:00')

In [269]:

for indx in range(len(df.CAMIS)):
   if df.iloc[indx].date == df[df.CAMIS == df.iloc[indx].CAMIS].date.max():
      df.iloc[indx].Latest_Inspection = 1

C:\Users\logic_switch\AppData\Local\Temp\ipykernel_14384\3048666006.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[indx].Latest_Inspection = 1


KeyboardInterrupt: 

In [ ]:
df.Latest_Inspection.sum()

0

In [ ]:
latest_inspection_dicts[41425803]



Timestamp('2023-01-19 00:00:00')

In [ ]:
latest_inspection_dicts

{50107551: Timestamp('1900-01-01 00:00:00'),
 50131697: Timestamp('1900-01-01 00:00:00'),
 50123820: Timestamp('1900-01-01 00:00:00'),
 50119514: Timestamp('1900-01-01 00:00:00'),
 50127198: Timestamp('1900-01-01 00:00:00'),
 50131632: Timestamp('1900-01-01 00:00:00'),
 50121462: Timestamp('1900-01-01 00:00:00'),
 50112539: Timestamp('1900-01-01 00:00:00'),
 50109522: Timestamp('1900-01-01 00:00:00'),
 50129927: Timestamp('1900-01-01 00:00:00'),
 41564948: Timestamp('2018-11-09 00:00:00'),
 50113426: Timestamp('1900-01-01 00:00:00'),
 50128218: Timestamp('1900-01-01 00:00:00'),
 50104825: Timestamp('1900-01-01 00:00:00'),
 50112870: Timestamp('1900-01-01 00:00:00'),
 50120379: Timestamp('1900-01-01 00:00:00'),
 50124520: Timestamp('1900-01-01 00:00:00'),
 50124855: Timestamp('1900-01-01 00:00:00'),
 50132225: Timestamp('1900-01-01 00:00:00'),
 50116143: Timestamp('1900-01-01 00:00:00'),
 50129141: Timestamp('1900-01-01 00:00:00'),
 50121299: Timestamp('1900-01-01 00:00:00'),
 50118414:

In [14]:
df['latestcrit'] = np.where(df['Critical'] == 1 and df.date == latest_inspection_dicts[df.CAMIS].max() , 1, 0)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [81]:
df.CAMIS

0         50107551
1         50131697
2         50123820
3         50119514
4         50127198
            ...   
209574    50052840
209575    50072454
209576    50093635
209577    40365882
209578    50111311
Name: CAMIS, Length: 209579, dtype: int64

In [ ]:
df.Latest_Inspection = df.CAMIS.map(latest_inspection_dicts)


In [ ]:
ISLATEST = pd.to_datetime(df['INSPECTION DATE']) == df.Latest_Inspection
df.isLatest = ISLATEST

In [ ]:
df['LatestandCrit'] = (df['isLatest'] ==True) & (df['Critical']==1 )

In [ ]:
df['Critical'].sum()

113768

In [ ]:
df[df.CAMIS == 40365882].Critical

3965      0
10210     0
41436     1
77401     0
97060     1
107194    1
116266    1
146949    0
209577    1
Name: Critical, dtype: int32

In [ ]:
abc = df.groupby('CAMIS')   

KeyError: 'Column not found: False'

In [ ]:
df.isLatest

0          True
1          True
2          True
3          True
4          True
          ...  
209574    False
209575    False
209576     True
209577    False
209578     True
Name: isLatest, Length: 209579, dtype: bool

In [83]:
len(df.CAMIS.unique())

28239

In [81]:
df.isLatest.sum()

74511

In [91]:
df[df.isLatest].groupby('CAMIS').max()

C:\Users\logic_switch\AppData\Local\Temp\ipykernel_14384\1787516273.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  df[df.isLatest].groupby('CAMIS').max()


,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE,INSPECTION DATE,ACTION,CRITICAL FLAG,...,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts,date,Critical,isLatest,LatestandCrit,NotLatestandCrit
CAMIS,,,,,,,,,,,,,,,,,,,,,
30075445,MORRIS PARK BAKE SHOP,Bronx,1007,MORRIS PARK AVENUE,10462.0,7188924968,Bakery Products/Desserts,2/3/2023,Establishment re-opened by DOHMH.,Not Critical,...,NaN,NaN,NaN,NaN,NaN,2023-02-03,1,True,True,False
30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,7/13/2022,Violations were cited in the following area(s).,Not Critical,...,NaN,NaN,NaN,NaN,NaN,2022-07-13,1,True,True,False
30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,351,WEST 57 STREET,10019.0,2122452912,Irish,1/4/2022,Violations were cited in the following area(s).,Not Critical,...,NaN,NaN,NaN,NaN,NaN,2022-01-04,1,True,True,False
40356018,RIVIERA CATERERS,Brooklyn,2780,STILLWELL AVENUE,11224.0,7183723031,American,2/1/2022,Violations were cited in the following area(s).,Critical,...,NaN,NaN,NaN,NaN,NaN,2022-02-01,1,True,True,False
40356483,WILKEN'S FINE FOOD,Brooklyn,7114,AVENUE U,11234.0,7184443838,Sandwiches,8/19/2022,Violations were cited in the following area(s).,Not Critical,...,NaN,NaN,NaN,NaN,NaN,2022-08-19,0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50132511,Girolamo's Pizza,Brooklyn,176,AVENUE O,11204.0,9178825572,NaN,1/1/1900,NaN,Not Applicable,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
50132516,HONEY CREPES,Manhattan,400,EAST 13 STREET,10009.0,6465719248,NaN,1/1/1900,NaN,Not Applicable,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
50132518,SUBWAY,Queens,10206,ATLANTIC AVE,11416.0,3478860495,NaN,1/1/1900,NaN,Not Applicable,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False


In [104]:
df[df.CAMIS == 40365882].NotLatestandCrit.max()

True

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE,INSPECTION DATE,ACTION,CODE,...,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts,date,Critical,isLatest,LatestandCrit,NotLatestandCrit
CAMIS,,,,,,,,,,,,,,,,,,,,,
30075445,MORRIS PARK BAKE SHOP,Bronx,1007,MORRIS PARK AVENUE,10462.0,7188924968,Bakery Products/Desserts,1/31/2023,Establishment Closed by DOHMH. Violations were...,06C,...,NaN,NaN,NaN,NaN,NaN,2023-01-31,1,False,False,True
30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,8/5/2021,Violations were cited in the following area(s).,04N,...,NaN,NaN,NaN,NaN,NaN,2021-08-05,1,False,False,True
30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,351,WEST 57 STREET,10019.0,2122452912,Irish,6/6/2019,Violations were cited in the following area(s).,10F,...,NaN,NaN,NaN,NaN,NaN,2019-06-06,0,False,False,False
40356018,RIVIERA CATERERS,Brooklyn,2780,STILLWELL AVENUE,11224.0,7183723031,American,11/8/2019,Violations were cited in the following area(s).,09B,...,NaN,NaN,NaN,NaN,NaN,2019-11-08,0,False,False,False
40356483,WILKEN'S FINE FOOD,Brooklyn,7114,AVENUE U,11234.0,7184443838,Sandwiches,7/30/2021,Violations were cited in the following area(s).,02I,...,NaN,NaN,NaN,NaN,NaN,2021-07-30,1,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50132511,Girolamo's Pizza,Brooklyn,176,AVENUE O,11204.0,9178825572,None,1/1/1900,None,None,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
50132516,HONEY CREPES,Manhattan,400,EAST 13 STREET,10009.0,6465719248,None,1/1/1900,None,None,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
50132518,SUBWAY,Queens,10206,ATLANTIC AVE,11416.0,3478860495,None,1/1/1900,None,None,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False


In [112]:
df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE,INSPECTION DATE,ACTION,...,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts,date,Critical,isLatest,LatestandCrit,NotLatestandCrit
0,50107551,Zazzys,Manhattan,73,GREENWICH AVENUE,10014.0,6465945909,NaN,1/1/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
1,50131697,NaN,Brooklyn,804,GRAND STREET,11211.0,3478078023,NaN,1/1/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
2,50123820,787 COFFEE,Manhattan,310,WEST 14 STREET,10014.0,9082308846,NaN,1/1/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
3,50119514,PEBBLE BAR,Manhattan,1240,AVE OF TH AMER,NaN,7189098636,NaN,1/1/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
4,50127198,Verdes Italian Eatery,Staten Island,4085,VICTORY BOULEVARD,10314.0,6313030934,NaN,1/1/1900,NaN,...,NaN,NaN,NaN,NaN,NaN,1900-01-01,0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209574,50052840,US FRIED CHICKEN & PIZZA,Brooklyn,129,DWIGHT STREET,11231.0,7186434662,Chicken,6/22/2022,Violations were cited in the following area(s).,...,NaN,NaN,NaN,NaN,NaN,2022-06-22,1,False,False,True
209575,50072454,DUNKIN,Manhattan,381,CANAL STREET,10013.0,2126191222,Donuts,8/22/2022,Violations were cited in the following area(s).,...,NaN,NaN,NaN,NaN,NaN,2022-08-22,1,False,False,True
209576,50093635,DOMINOS,Brooklyn,1544,BROADWAY,11221.0,5166038816,Pizza,8/8/2022,Violations were cited in the following area(s).,...,NaN,NaN,NaN,NaN,NaN,2022-08-08,0,True,False,False
209577,40365882,NEW YORK PIZZA SUPREMA,Manhattan,413,8 AVENUE,10001.0,2125948939,Pizza,2/8/2022,Violations were cited in the following area(s).,...,NaN,NaN,NaN,NaN,NaN,2022-02-08,1,False,False,True


In [89]:
abc.LatestandCrit.sum()


20404

In [90]:
abc.NotLatestandCrit.sum()

0

In [283]:
Critication = {}
BigCritication = {}
for name in Name_list:
    Critication[name] = df.NotLatestandCrit[df.CAMIS == name].max()
    BigCritication[name] = df.LatestandCrit[df.CAMIS == name].max()

AttributeError: 'DataFrame' object has no attribute 'NotLatestandCrit'

In [117]:
df.NotLatestandCrit

0         False
1         False
2         False
3         False
4         False
          ...  
209574     True
209575     True
209576    False
209577     True
209578    False
Name: NotLatestandCrit, Length: 209579, dtype: bool

{50107551: False,
 50131697: False,
 50123820: False,
 50119514: False,
 50127198: False,
 50131632: False,
 50121462: False,
 50112539: False,
 50109522: False,
 50129927: False,
 41564948: False,
 50113426: False,
 50128218: False,
 50104825: False,
 50112870: False,
 50120379: False,
 50124520: False,
 50124855: False,
 50132225: False,
 50116143: False,
 50129141: False,
 50121299: False,
 50118414: False,
 50008108: True,
 50108007: False,
 41089821: True,
 50105922: False,
 50129206: False,
 50125244: False,
 50115585: False,
 50129407: False,
 50118566: False,
 50130580: False,
 50106095: False,
 50128131: False,
 50126683: False,
 50132112: False,
 50122701: False,
 50124519: False,
 50118608: False,
 50131334: False,
 50127688: False,
 50121100: False,
 50123083: False,
 50105337: False,
 50124763: False,
 50119329: False,
 50125888: False,
 50131922: False,
 50123419: False,
 50106042: False,
 50119637: False,
 41600677: True,
 50103832: False,
 50105750: False,
 50126340: Fa

In [128]:
pastCrit = {}
nowCrit = {}
for name in Name_list:
    pastCrit[name] = df.NotLatestandCrit[df.CAMIS == name].max()
    nowCrit[name] = df.LatestandCrit[df.CAMIS == name].max()

df['Now_Crit'] = df.CAMIS.map(nowCrit)
df['Past_Crit'] = df.CAMIS.map(pastCrit)


In [187]:
abc = df[df.isLatest == True].groupby('CAMIS').first()

In [188]:
finalVariables = ['Now_Crit','CUISINE', 'BORO', 'ZIPCODE', 'Community Board', 'Past_Crit']
to_convert = ['ZIPCODE', 'Community Board']
abc[to_convert] = abc[to_convert].astype('category')
abc = abc.drop(abc[abc.date== '1900-01-01'].index)
abc = abc[finalVariables]

In [189]:
abc.Now_Crit.value_counts()

True     20404
False     3477
Name: Now_Crit, dtype: int64

In [190]:
abc.Past_Crit.value_counts()

True     18689
False     5192
Name: Past_Crit, dtype: int64

In [171]:
df.date == '1900-01-01' and df.Past_Crit == False


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [174]:
df[(df.date == '1900-01-01') & (df.Now_Crit == True)]

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE,INSPECTION DATE,ACTION,...,Borough Boundaries,City Council Districts,Police Precincts,date,Critical,isLatest,LatestandCrit,NotLatestandCrit,Past_Crit,Now_Crit


In [203]:
abc.isnull().sum(axis = 0).sort_values(ascending = False)

Community Board    451
ZIPCODE            361
Now_Crit             0
CUISINE              0
BORO                 0
Past_Crit            0
dtype: int64

## Now its time for imputation

In [284]:
impute_variables = ['CAMIS', 'Community Board', 'Longitude', 'Latitude']
df[to_convert] = df[to_convert].astype('category')
impute_df = df[impute_variables].groupby('CAMIS').first()

In [285]:
impute_df

,Community Board,Longitude,Latitude
CAMIS,,,
30075445,211.0,-73.855972,40.848231
30112340,309.0,-73.962081,40.662652
30191841,104.0,-73.984310,40.767326
40356018,313.0,-73.982090,40.579920
40356483,318.0,-73.906989,40.620112
...,...,...,...
50132511,311.0,-73.978401,40.610812
50132516,103.0,-73.982742,40.730499
50132518,409.0,-73.842541,40.689932


In [286]:
from sklearn.impute import KNNImputer


imputer = KNNImputer(n_neighbors=2, weights="uniform")
a=imputer.fit_transform(impute_df)

In [292]:
df_imputed = pd.DataFrame(a, columns= list(impute_df), index= impute_df.index)

In [293]:
df_imputed

,Community Board,Longitude,Latitude
CAMIS,,,
30075445,211.0,-73.855972,40.848231
30112340,309.0,-73.962081,40.662652
30191841,104.0,-73.984310,40.767326
40356018,313.0,-73.982090,40.579920
40356483,318.0,-73.906989,40.620112
...,...,...,...
50132511,311.0,-73.978401,40.610812
50132516,103.0,-73.982742,40.730499
50132518,409.0,-73.842541,40.689932


In [294]:
df_imputed.rename(columns = {'Community Board':'CBoard'}, inplace = True)

In [296]:
Community_Board_Imputed = {}
for name in Name_list:
    Community_Board_Imputed[name] = df_imputed.CBoard[name]
    

In [298]:
Community_Board_Imputed

{50107551: 102.0,
 50131697: 301.0,
 50123820: 102.0,
 50119514: 414.0,
 50127198: 502.0,
 50131632: 303.0,
 50121462: 401.0,
 50112539: 102.0,
 50109522: 102.0,
 50129927: 306.0,
 41564948: 105.0,
 50113426: 104.0,
 50128218: 107.0,
 50104825: 414.0,
 50112870: 105.0,
 50120379: 103.0,
 50124520: 205.0,
 50124855: 207.0,
 50132225: 408.0,
 50116143: 301.0,
 50129141: 105.0,
 50121299: 106.0,
 50118414: 102.0,
 50008108: 102.0,
 50108007: 106.0,
 41089821: 414.0,
 50105922: 315.0,
 50129206: 307.0,
 50125244: 203.0,
 50115585: 409.0,
 50129407: 407.0,
 50118566: 303.0,
 50130580: 406.0,
 50106095: 105.0,
 50128131: 402.0,
 50126683: 304.0,
 50132112: 402.0,
 50122701: 102.0,
 50124519: 205.0,
 50118608: 307.0,
 50131334: 102.0,
 50127688: 205.0,
 50121100: 401.0,
 50123083: 210.0,
 50105337: 104.0,
 50124763: 303.0,
 50119329: 104.0,
 50125888: 307.0,
 50131922: 303.0,
 50123419: 402.0,
 50106042: 401.0,
 50119637: 104.0,
 41600677: 102.0,
 50103832: 112.0,
 50105750: 211.0,
 50126340:

In [305]:
df['Community Board'] = df.CAMIS.map(Community_Board_Imputed)

In [301]:
len(df_imputed['CBoard'].unique()) 

70

In [302]:
#assert len(df_imputed.ZIPCODE.unique()) == len(df.ZIPCODE.unique())
assert len(df_imputed['Community Board'].unique()) == len(df['Community Board'].unique())


KeyError: 'Community Board'

In [281]:
len(df.CAMIS.unique())

28239

0